In [ ]:
%pylab inline

import spot
import PatternStimulus as ps
import pickle
import sequence
reload(sequence)
import random
from time_shifter import duration_shifter
from copy import deepcopy

In [ ]:
with open('11941.pickle','rb') as fn:
    sess, spots = pickle.load(fn)

In [ ]:
ptn='A10_B50_C90_D130_E170_F210-14'

nowSess=sess[-1]
xy,timing,intensities,grouptimes,rand_args=ps.get_pattern_params(spots['list'],ptn,nowSess) 

fixed_spotsize = nowSess['pre']['gridsize']
spotsizes=[fixed_spotsize for x in range(len(xy))] #TODO: variable spot size per spot
isProbe = 1
rig='Polygon'
seq_obj=sequence.Sequence2(spots=xy,
                          timing=timing,
                          isProbe=isProbe,
                          rig=rig,
                          spotsizes=spotsizes,
                          label=ptn,
                          mode='grid',
                          intensities=intensities,
                          grouptimes=grouptimes,
                          rand_args=rand_args)

if seq_obj.isRandomSequence:
    seq_obj.rand_spotlist = spots['list']['R']




In [ ]:
target=deepcopy(seq_obj)
tptn = {}
for s in target.spotlist:
    tptn[tuple(s.xy)]=s.timing

onsets = [t[0] for t in tptn.values()]
onsets.sort()

In [ ]:
spot2pos = {}
for s,t in tptn.items():
    pos = onsets.index(t[0])
    spot2pos[s] = pos 

In [ ]:
replaceVec_all = []
shiftVec_all = []
nspots=len(tptn)
for i in range(2000):
    seq_obj.randomize()
    replaceVec = [0]*nspots
    shiftVec = [0]*nspots

    ptn = {}
    for s in seq_obj.spotlist:
        ptn[tuple(s.xy)]=s.timing

    for tgt_s,tgt_t in tptn.items():
        pos = spot2pos[tgt_s]
        if tgt_s not in ptn:
            replaceVec[pos] = 1
        else:
            t = ptn[tgt_s]
            shiftVec[pos] = t[0]-tgt_t[0]

    replaceVec_all.append(replaceVec)
    shiftVec_all.append(shiftVec)

In [ ]:
replaceVec_all = np.array(replaceVec_all)
shiftVec_all = np.array(shiftVec_all)

vec=shiftVec_all[:,6]
plt.hist(vec[vec!=0])
plt.xlim([-90,90])
#plt.ylim([0,600])